<div style='text-align: center;'>
<img src="images/math60082-banner.png" alt="image" width="80%" height="auto">
</div>

# Lab Class - Week 3
## Dr P. V. Johnson
## Department of Mathematics

- How to solve problems with code
- Demo 2.1 - Integrating a normal distribution
- Demo 2.2 - How to evaluate the efficiency of code
- Demo 2.3 - Coursework Example